## 2.5 SQLAlchemy

<p style="font-size:120%">
Déclaration du mapping objet-relationnel, méthode impérative ou déclarative
</p>

### Méthode impérative

In [1]:
### imperative

from sqlalchemy import create_engine
from sqlalchemy import MetaData, Table, Column, Integer, String
from sqlalchemy.orm import sessionmaker, mapper

# Description de la table
user_table = Table(
    "user",
    MetaData(),
    Column('id',Integer, primary_key=True),
    Column('pseudo',String(16), nullable=False),
    Column('email',String(255), nullable=False),
    Column('pwd',String(32), nullable=False) 
)

# Objet métier
class User2:
    
    def __init__(self,pseudo,email,pwd):
        self.pseudo = pseudo
        self.email = email
        self.pwd = pwd
        self.id = None
        
    def __str__(self):
        return '[{}] {}: {}'.format(self.id,self.pseudo,self.email)

# Mapper - correspondance entre l'objet métier et la description de la table
m = mapper(User2, user_table)

# Création d'un objet métier
import random
import string

def get_random_string(length):
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for i in range(length))

# Insert
pseudo = get_random_string(3)
user = User2(pseudo=pseudo, email='{}@gmail.com'.format(pseudo), pwd=get_random_string(8))

<div style="font-size: 120%" id="mapperlink">
L'objet métier comporte un lien vers le mapper :
</div>

In [2]:
from sqlalchemy.orm import  object_mapper

assert(object_mapper(user) == m)
assert(user._sa_class_manager.mapper == m)

<div style="font-size: 120%">
On peut donc l'enregistrer dans la base :
</div>

In [3]:
# Moteur d'accès à la base sqlalchemy.sqlite
engine = create_engine('sqlite:///sqlalchemy.sqlite')

# Session
Session = sessionmaker(bind=engine)
session = Session()

# Insert
session.add(user)
session.commit()

print(user.id, user.pseudo, user.email, user.pwd)

20 mye mye@gmail.com tvvvefsk


<div style="font-size: 120%">
La relation avec le mapper existe au niveau de la classe de l'objet métier :
</div>

In [4]:
assert(User2._sa_class_manager.mapper == m)

<div style="font-size: 120%">
ce qui permet d'effectuer des requêtes comme celle-ci :
</div>

In [15]:
# Select
users = session.query(User2).all()
for user in users:
    print(user)

[1] rde: Raymond.Deubaze@gmail.com
[2] nxq: nxq@gmail.com
[3] ztj: ztj@gmail.com
[4] plm: plm@gmail.com
[5] rtv: rtv@gmail.com
[6] ofp: ofp@gmail.com
[7] kae: kae@gmail.com
[8] vbs: vbs@gmail.com
[9] jty: jty@gmail.com
[10] jbt: jean.bonnot@ec-lyon.fr
[11] phh: phh@gmail.com
[12] xil: xil@gmail.com
[13] qfk: qfk@gmail.com
[14] ijt: ijt@gmail.com
[15] yhw: yhw@gmail.com
[16] upc: upc@gmail.com
[17] nym: nym@gmail.com
[18] lgu: lgu@gmail.com
[19] obu: obu@gmail.com
[20] mye: mye@gmail.com


### Méthode déclarative

In [6]:
### declarative

from sqlalchemy import Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

# Moteur d'accès à la base sqlalchemy.sqlite
engine = create_engine('sqlite:///sqlalchemy.sqlite')

# Classe de base
Base = declarative_base()
Base.metadata.bind = engine

# Définition de la classe correspondant à la table "user"
class User(Base):
    __tablename__ = 'user'

    id = Column(Integer, primary_key=True)
    pseudo = Column(String(16), nullable=False)
    email = Column(String(255), nullable=False)
    pwd = Column(String(32), nullable=False) 

<div style="font-size: 120%">
Ici, la table est indiquée de manière explicite dans le corps de la classe métier, et le mapper est automatiquement instancié et lié avec la classe mère :
</div>

In [7]:
user = User(pseudo='abc', email='abc@gmail.com', pwd='hello')

In [8]:
from sqlalchemy.orm import  object_mapper

assert(object_mapper(user) == user.__mapper__)
assert(user.__tablename__ == 'user')

assert User.__mapper__ == user.__mapper__

<div style="font-size: 120%" id="session">
Quelle que soit la méthode utilisée pour décrire le mapping (impérative ou déclarative) les accès
à la base de données se font ensuite via une instance de la classe <code>Session</code>:
</div>

In [12]:
# Session
Session = sessionmaker(bind=engine)
session = Session()
print(session)

In [13]:
# Requête
users = session.query(User).all()
for user in users:
    print(user.pseudo,user.email)

rde Raymond.Deubaze@gmail.com
nxq nxq@gmail.com
ztj ztj@gmail.com
plm plm@gmail.com
rtv rtv@gmail.com
ofp ofp@gmail.com
kae kae@gmail.com
vbs vbs@gmail.com
jty jty@gmail.com
jbt jean.bonnot@ec-lyon.fr
phh phh@gmail.com
xil xil@gmail.com
qfk qfk@gmail.com
ijt ijt@gmail.com
yhw yhw@gmail.com
upc upc@gmail.com
nym nym@gmail.com
lgu lgu@gmail.com
obu obu@gmail.com
mye mye@gmail.com
